In [13]:
import requests
import pandas as pd
import time
import requests
import requests.auth
import praw
import pickle
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from nltk.stem import PorterStemmer, SnowballStemmer, LancasterStemmer
import nltk


#saving
#pickle.dump(master_df, open("master_df.pickle", "wb" ))

#reading df
#submissions_df=pickle.load(open("submissions_df.pickle", "rb"))
#comments_df=pickle.load(open("comments_df.pickle", "rb"))



In [16]:
# full dataframe of comments used to construct the dfs and dicts below

pd.DataFrame(comments_df['body'])

,body
0,I comment regardless. Those moons aren't farmi...
1,Yeah precisely. I don’t want anything to kill ...
2,Tinyman is the single most important app for A...
3,Another one of these “crash is incoming” posts...
4,Hindsight is 20/20. This is safe considering y...
...,...
1814617,Brace yourselves; more adoption incoming! Bull...
1814618,But it’s unfair for us to skip out on taxes vs...
1814619,[removed]
1814620,[removed]


# Sentiment analysis


## read in pickled data

In [7]:
# uncleaded dict for sentiment analysis
uncleaned_dict=pickle.load(open("uncleaned_dict.pickle", "rb"))

# cleaned dict for topic modeling
a_dict=pickle.load(open("cleaned_dict.pickle", "rb"))

# full dataframe 
sample_df=pickle.load(open("processed_df.pickle", "rb"))

#list of cryptos
cryp_list = pickle.load(open("cryp_list.pickle", "rb"))


## Cleaned dict containing each crypto as a key and values as dfs with all sentences that mention the crypto 

In [17]:
# eg any sentence that contains bitcoin or btc (the symbol)
a_dict['btc']


,0
10,tell mine scam buy bitcoin
19,alt bit coin nice alt bitcoin ahhhh
29,possibl btc market cap possibl
113,yeah good reason hold bitcoin bag sell
137,like prefil donat btc think crypto hodler rich...
...,...
1334314,yes mean help actual differ shard zk rollup si...
1334342,real want ltcdoge btc miner
1334349,bitcoin alt coin alt coin dispos asset
1334368,promis btc manag crypto


## sentiment analysis for each crypt (sampled at 500 comments bc processing time)

In [117]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

analyzer = SentimentIntensityAnalyzer() 


#get sentiment for each crypto using uncleaned data (to pick up on punctuation and capitalization etc)
for key in uncleaned_dict:
    if len(uncleaned_dict[key]['body'])>5000:
        print(key, analyzer.polarity_scores(uncleaned_dict[key]['body'].sample(500)))

btc {'neg': 0.074, 'neu': 0.808, 'pos': 0.118, 'compound': 1.0}
eth {'neg': 0.062, 'neu': 0.806, 'pos': 0.132, 'compound': 1.0}
usdt {'neg': 0.081, 'neu': 0.803, 'pos': 0.115, 'compound': 1.0}
sol {'neg': 0.065, 'neu': 0.81, 'pos': 0.125, 'compound': 1.0}
ada {'neg': 0.069, 'neu': 0.791, 'pos': 0.14, 'compound': 1.0}
dot {'neg': 0.044, 'neu': 0.812, 'pos': 0.145, 'compound': 1.0}
doge {'neg': 0.074, 'neu': 0.797, 'pos': 0.129, 'compound': 1.0}
shib {'neg': 0.078, 'neu': 0.791, 'pos': 0.132, 'compound': 1.0}
cro {'neg': 0.045, 'neu': 0.805, 'pos': 0.15, 'compound': 1.0}
matic {'neg': 0.044, 'neu': 0.812, 'pos': 0.144, 'compound': 1.0}
algo {'neg': 0.046, 'neu': 0.802, 'pos': 0.152, 'compound': 1.0}
link {'neg': 0.067, 'neu': 0.812, 'pos': 0.121, 'compound': 1.0}
xlm {'neg': 0.048, 'neu': 0.821, 'pos': 0.131, 'compound': 1.0}
vet {'neg': 0.056, 'neu': 0.808, 'pos': 0.136, 'compound': 1.0}
etc {'neg': 0.076, 'neu': 0.794, 'pos': 0.13, 'compound': 1.0}
xmr {'neg': 0.08, 'neu': 0.792, 'pos'

# Topic Modeling

In [52]:
# Function to display the top n terms in each topic
def display_topics(model, feature_names, no_top_words, topic_names = None): 
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix + 1)
        else:
            print("\nTopic: ", topic_names[ix])
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
    print("\n")
    return model, feature_names, no_top_words

## LSA for all data

In [53]:
#sample the whole df (poorly named sample_df)
mvp_df=pd.DataFrame(sample_df).sample(100)

In [58]:
cv = CountVectorizer(stop_words='english', ngram_range = (1,2))
#applying fit_transform on review
X_vect = cv.fit_transform(mvp_df[0]).toarray()
#converting to df and naming columns and 
X_vect_df=pd.DataFrame(X_vect,index=mvp_df,columns=cv.get_feature_names_out())


lsa = TruncatedSVD(2)
doc_topic = lsa.fit_transform(X_vect_df)
doc_topic_df=pd.DataFrame(doc_topic.round(5),
             index = pd.Series(mvp_df[0]),
             columns = ["component_1","component_2" ])
lsa.explained_variance_ratio_

### YOUR CODE HERE
topic_term=lsa.components_
topic_term_df=pd.DataFrame(topic_term.round(3), 
             index = ["component_1", "component_2"],
             columns = cv.get_feature_names_out())



output=display_topics(lsa,cv.get_feature_names_out(),10)


doc_topic_df=pd.DataFrame(doc_topic.round(5),
             index = pd.Series(mvp_df[0]),
             columns = ["component_1","component_2" ])

print(lsa.explained_variance_ratio_)

topic_term_df
output
doc_topic_df








Topic  1
margin, usdt, borrow, margin ratio, ftm, ratio, collater, trade, allow, loan

Topic  2
peopl, number, attack, context, theori, year, plan, discuss, chanc, low


[0.14810261 0.05389051]


,component_1,component_2
0,,
random look funni haha dog coin avax,0.00576,0.13105
deposit usdt kucoin exchang issu send kucoin purchas issu,0.38954,0.09392
left door slight ajar,0.06127,-0.00247
read post gifgiphydhqthhvqoiz,0.00125,0.00595
holdin game token,0.00193,0.10079
...,...,...
gotten bed,0.00000,0.00000
chanc year peopl chase coin chanc go week baffl sure invest coin hope best major btc eth,0.02053,0.93175
btc eth,0.00438,0.07340


# Comments

### So far, I've rounded up my data, created a dict containing each crypto as the key and values as a df containing all comments. The next step is trying out different dimensionality reduction methods for each crypto's comments. I've created a function but it's been crashing my system, so I'll need to troubleshoot. I'm also running a sentiment analysis for 5000 comments for each crypto to see what gauge r/CryptoCurrency's views on each crypto. 